<a href="https://colab.research.google.com/github/usneek/Multilingual-Conversational-Question-Answering/blob/main/Roberta_ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U tensorflow-text
!pip install -q tf-models-official

     |████████████████████████████████| 4.4 MB 7.1 MB/s 
     |████████████████████████████████| 1.8 MB 7.1 MB/s 
     |████████████████████████████████| 1.1 MB 43.0 MB/s 
     |████████████████████████████████| 1.2 MB 45.3 MB/s 
     |████████████████████████████████| 99 kB 9.1 MB/s 
     |████████████████████████████████| 90 kB 8.7 MB/s 
     |████████████████████████████████| 352 kB 68.8 MB/s 
     |████████████████████████████████| 213 kB 53.7 MB/s 
     |████████████████████████████████| 37.1 MB 51 kB/s 
     |████████████████████████████████| 636 kB 56.0 MB/s 
     |████████████████████████████████| 43 kB 2.0 MB/s 


In [ ]:
import pandas as pd
import numpy as np
np.random.seed(0)
from keras.regularizers import l2
from keras.models import Model, Sequential
from keras.layers import Dense, Input, Dropout, LSTM, Activation, Conv1D, Flatten
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.metrics import f1_score
np.random.seed(1)
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization
import matplotlib.pyplot as plt
import re
import string
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
tf.get_logger().setLevel('ERROR')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json
data_file = open('drive/My Drive/Conversational Question Answering System Code Notebooks/data.json',encoding='utf-8').read()
intents = json.loads(data_file)

In [ ]:
sentence=[]
for intent in intents['intents']:
    for pattern in intent['patterns']:
        sentence.append(pattern)

In [ ]:
classes=[]
for intent in intents['intents']:
  for pattern in intent['patterns']:
    classes.append(intent['tag'])

In [ ]:
print(sentence)

['सीज़ेंट्रिक्स कॉलबैक', 'सीजेड कैक्लबैक', 'सीज़ेंट्रिक्स कॉल बैक', 'C केंद्र कॉल बैक', 'सीज़ेंट्रिक्स कॉल बैक', 'वापस कॉल करें', 'क्या आप कॉल बैक सुविधा प्रदान करते हैं', 'कॉल बैक सॉल्यूशन', 'एक कॉल बैक समाधान की तलाश में', 'कॉल बैक सेवा की आवश्यकता है', 'कॉल बैक सॉल्यूशन है', 'सीज़ेंट्रिक्स कॉलबैक क्या है', 'क्या आप मुझे सीज़ेंट्रिक्स कॉलबैक समाधान में मदद कर सकते हैं', 'क्या आप मुझे सीज़ेंट्रिक्स कॉल बैक सॉल्यूशन में मदद कर सकते हैं', 'क्या आप मुझे कॉल बैक समाधान के बारे में बता सकते हैं', 'ठीक है धन्यवाद मुझे सीज़ेंट्रिक्स कॉलबैक के बारे में बताओ', 'धन्यवाद, मुझे कॉलबैक के बारे में बताएं', 'थैंक्यू, कॉल बैक क्या है', 'अच्छा, क्या आप मुझे कॉलबैक के बारे में बता सकते हैं?', 'धन्यवाद मुझे सीज़ेंट्रिक्स कॉलबैक के बारे में बताएं', ' सीज़ेंट्रिक्स हेल्पडेस्क', 'सीज़ेंट्रिक्सहेल्पडेस्क', 'सीज़ेंट्रिक्स मदद', 'सीज़ेंट्रिक्स हेल्पडेस्क', 'मदद', 'सहायता केंद्र', 'हेल्प डेस्क', 'सीज़ेंट्रिक्स हेल्पडेस्क', 'क्या आप हेल्पडेस्क सुविधा प्रदान करते हैं', 'एक हेल्प डेस्क समाधान की तलाश में', 'मदद ड

In [ ]:
print(classes)

['cz_callback', 'cz_callback', 'cz_callback', 'cz_callback', 'cz_callback', 'cz_callback', 'cz_callback', 'cz_callback', 'cz_callback', 'cz_callback', 'cz_callback', 'cz_callback', 'cz_callback', 'cz_callback', 'cz_callback', 'cz_callback', 'cz_callback', 'cz_callback', 'cz_callback', 'cz_callback', 'cz_helpdesk', 'cz_helpdesk', 'cz_helpdesk', 'cz_helpdesk', 'cz_helpdesk', 'cz_helpdesk', 'cz_helpdesk', 'cz_helpdesk', 'cz_helpdesk', 'cz_helpdesk', 'cz_helpdesk', 'cz_helpdesk', 'cz_helpdesk', 'cz_helpdesk', 'cz_helpdesk', 'cz_helpdesk', 'cz_helpdesk', 'cz_helpdesk', 'cz_helpdesk', 'crm_application', 'crm_application', 'crm_application', 'crm_application', 'crm_application', 'crm_application', 'crm_application', 'crm_application', 'crm_application', 'crm_application', 'crm_application', 'crm_application', 'crm_application', 'crm_application', 'crm_application', 'non_voice_process', 'non_voice_process', 'non_voice_process', 'non_voice_process', 'non_voice_process', 'non_voice_process', 'no

In [ ]:
from numpy import array
from numpy import argmax
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
encoder = LabelEncoder()
encoder.fit(classes)
encoded_Y = encoder.transform(classes)
encoded_Y=np_utils.to_categorical(encoded_Y)

In [ ]:
print(encoded_Y)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
X=[]
for i in sentence:
  X.append(i)

In [ ]:
print(X)

['सीज़ेंट्रिक्स कॉलबैक', 'सीजेड कैक्लबैक', 'सीज़ेंट्रिक्स कॉल बैक', 'C केंद्र कॉल बैक', 'सीज़ेंट्रिक्स कॉल बैक', 'वापस कॉल करें', 'क्या आप कॉल बैक सुविधा प्रदान करते हैं', 'कॉल बैक सॉल्यूशन', 'एक कॉल बैक समाधान की तलाश में', 'कॉल बैक सेवा की आवश्यकता है', 'कॉल बैक सॉल्यूशन है', 'सीज़ेंट्रिक्स कॉलबैक क्या है', 'क्या आप मुझे सीज़ेंट्रिक्स कॉलबैक समाधान में मदद कर सकते हैं', 'क्या आप मुझे सीज़ेंट्रिक्स कॉल बैक सॉल्यूशन में मदद कर सकते हैं', 'क्या आप मुझे कॉल बैक समाधान के बारे में बता सकते हैं', 'ठीक है धन्यवाद मुझे सीज़ेंट्रिक्स कॉलबैक के बारे में बताओ', 'धन्यवाद, मुझे कॉलबैक के बारे में बताएं', 'थैंक्यू, कॉल बैक क्या है', 'अच्छा, क्या आप मुझे कॉलबैक के बारे में बता सकते हैं?', 'धन्यवाद मुझे सीज़ेंट्रिक्स कॉलबैक के बारे में बताएं', ' सीज़ेंट्रिक्स हेल्पडेस्क', 'सीज़ेंट्रिक्सहेल्पडेस्क', 'सीज़ेंट्रिक्स मदद', 'सीज़ेंट्रिक्स हेल्पडेस्क', 'मदद', 'सहायता केंद्र', 'हेल्प डेस्क', 'सीज़ेंट्रिक्स हेल्पडेस्क', 'क्या आप हेल्पडेस्क सुविधा प्रदान करते हैं', 'एक हेल्प डेस्क समाधान की तलाश में', 'मदद ड

In [ ]:
!pip install sentence-transformers

     |████████████████████████████████| 78 kB 3.8 MB/s 
     |████████████████████████████████| 2.9 MB 19.7 MB/s 
     |████████████████████████████████| 3.3 MB 41.0 MB/s 
     |████████████████████████████████| 56 kB 2.1 MB/s 
     |████████████████████████████████| 895 kB 71.6 MB/s 
     |████████████████████████████████| 109 kB 66.8 MB/s 
     |████████████████████████████████| 546 kB 55.8 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=121000 sha256=2f51dddcb720aa9ae51718c615bb629a612a275bd0cd557a52b4150d0d77afc6
  Stored in directory: /root/.cache/pip/wheels/90/f0/bb/ed1add84da70092ea526466eadc2bfb197c4bcb8d4fa5f7bad
Successfully built sentence-transformers


In [ ]:
from sentence_transformers import SentenceTransformer
sbert_model = SentenceTransformer('xlm-roberta-base')

Downloading:   0%|          | 0.00/345 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.77k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/512 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
sentence_embedding=sbert_model.encode(X)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2227: UserWarning: `max_length` is ignored when `padding`=`True`.
  warnings.warn("`max_length` is ignored when `padding`=`True`.")


In [ ]:
model = Sequential()
model.add(Dense(512, input_shape=(sentence_embedding.shape[1],), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(33, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 
hist = model.fit(sentence_embedding,encoded_Y, epochs=600, batch_size=10, verbose=1)
model.save('chatbot_model.h5', hist)
print("model created")

Epoch 1/600
53/53 [==============================] - 1s 4ms/step - loss: 3.6923 - accuracy: 0.0302
Epoch 2/600
53/53 [==============================] - 0s 4ms/step - loss: 3.4934 - accuracy: 0.0491
Epoch 3/600
53/53 [==============================] - 0s 5ms/step - loss: 3.4482 - accuracy: 0.0509
Epoch 4/600
53/53 [==============================] - 0s 5ms/step - loss: 3.3732 - accuracy: 0.0849
Epoch 5/600
53/53 [==============================] - 0s 4ms/step - loss: 3.3119 - accuracy: 0.0925
Epoch 6/600
53/53 [==============================] - 0s 4ms/step - loss: 3.2193 - accuracy: 0.1283
Epoch 7/600
53/53 [==============================] - 0s 4ms/step - loss: 3.1559 - accuracy: 0.1226
Epoch 8/600
53/53 [==============================] - 0s 4ms/step - loss: 3.0235 - accuracy: 0.1698
Epoch 9/600
53/53 [==============================] - 0s 4ms/step - loss: 2.9172 - accuracy: 0.1849
Epoch 10/600
53/53 [==============================] - 0s 4ms/step - loss: 2.8197 - accuracy: 0.2113
Epoch 11/

In [ ]:
import random
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

In [ ]:
final=['','affirm','crm_application','cz_acd','cz_bar','cz_bot','cz_callback',
 'cz_callblast','cz_chat','cz_conference','cz_dialer','cz_guide',
 'cz_helpdesk','cz_ivr','cz_lms','cz_magic_call','cz_missed_call',
 'cz_mobile','cz_screen_logger','cz_voice_logger','deny','greet','home',
 'manual_dialer','non_voice_process','predictive_dialer','preview_dialer',
 'product','progressive_dialer','self_service_solutions','thankyou',
 'voice_process','work_force_management']

In [ ]:
def chatbot_response(text):
  text=[text]
  em=sbert_model.encode(text)
  res = model.predict(em)[0]
  ERROR_THRESHOLD = 0.3
  results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
  results.sort(key=lambda x: x[1], reverse=True)
  return_list = []
  for r in results:
      return_list.append({"intent": final[r[0]], "probability": str(r[1])})
  res = getResponse(return_list, intents)
  print(res)
 

In [ ]:
chatbot_response("सीज़ेंट्रिक्स कॉलबैक")

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2227: UserWarning: `max_length` is ignored when `padding`=`True`.
  warnings.warn("`max_length` is ignored when `padding`=`True`.")


सीज़ेंट्रिक्स कॉलबैक समाधान आपके भावी ग्राहकों को लंबे समय तक कॉल पर प्रतीक्षा नहींकरने में मदद करता है। उन्हें तुरंत कॉल करें या उनकी सुविधानुसार कॉल शेड्यूल  करें। आगंतुक या ग्राहक अपना संपर्क नंबर प्रदान कर सकते हैं और संबंधित एजेंटवापस कॉल करेंगे। यह आपके ग्राहक के ड्रॉपआउट दरों को बचाता है। अधिक जानकारी केलिए लिंक पर जाएं


In [ ]:
chatbot_response("kya aapke paas dialer ka progressive mode hai")

सीज़ेंट्रिक्स कॉन्फ़्रेंस स्वचालित डायलर, अनुस्मारक सूचनाएं और रिकॉर्ड मीटिंग जैसी कई उन्नत सुविधाएँ प्रदान करता है। सभी कॉल पासवर्ड से सुरक्षित हैं। एसएमएस और ईमेल एकीकरण कुशल अधिसूचना क्षमता प्रदान करता है। अधिक जानकारी के लिए यात्रा लिंक,<a href='https://www.c-zentrix.com/products/cz-conferencing-bridge'>यहाँ क्लिक करें</a>


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2227: UserWarning: `max_length` is ignored when `padding`=`True`.
  warnings.warn("`max_length` is ignored when `padding`=`True`.")


In [ ]:
chatbot_response("हांजी")

आपकी प्रतिक्रिया के लिए आपका धन्यवाद। होम पर जाए


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2227: UserWarning: `max_length` is ignored when `padding`=`True`.
  warnings.warn("`max_length` is ignored when `padding`=`True`.")


In [ ]:
chatbot_response("hello")

मैं आपकी मदद कर सकता हूं विक्रय संबंधी जानकारी प्रोडक्ट की जानकारी ग्राहक सहायता


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2227: UserWarning: `max_length` is ignored when `padding`=`True`.
  warnings.warn("`max_length` is ignored when `padding`=`True`.")


In [ ]:
chatbot_response("मिस्ड कॉल के बारे में बताओ")

सीज़ेड मिस्ड कॉल समाधान उत्पन्न लीड की तरह बहुत कुछ कर सकता है, अंत उपयोगकर्ताओं को कनेक्ट करने, संख्याओं की पुष्टि करने, प्रतिक्रिया एकत्र करने, वोटों का संचालन करने और यहां तक कि प्रतियोगिता चलाने के लिए एक निःशुल्क मंच प्रदान करता है। उपयोगकर्ता बस एक मिस्ड कॉल कर सकता है और हमारे समाधान द्वारा आराम का ध्यान रखा जाता है। अधिक जानकारी के लिए यात्रा लिंक,<a href='https://www.czentrix.com/products/czmissedcallservice'>यहाँ क्लिक करें</a>


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2227: UserWarning: `max_length` is ignored when `padding`=`True`.
  warnings.warn("`max_length` is ignored when `padding`=`True`.")


In [ ]:
chatbot_response("tell me about dialers")

सीज़ेंट्रिक्स मोबाइल, मोबाइल पहला समाधान है जो लैपटॉप / डेस्कटॉप / PRI / लैंडलाइन पर एजेंटों की निर्भरता को हटाकर और उनके स्मार्टफ़ोन पर टेलीफोनी लाकर अधिक गतिशीलता को प्रोत्साहित करता है। यह बिक्री, सहायता और संग्रह में वितरित कार्यबल के लिए काफी हद तक उपयोगी है। अधिक जानकारी के लिए लिंक पर जाएं,<a href='https://www.czentrix.com/products/czmobile'>यहाँ क्लिक करें</a>


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2227: UserWarning: `max_length` is ignored when `padding`=`True`.
  warnings.warn("`max_length` is ignored when `padding`=`True`.")


In [ ]:
chatbot_response("ਮੈਨੂੰ ਡਾਇਲਰ ਬਾਰੇ ਦੱਸੋ")

सीज़ेंट्रिक्स कॉन्फ़्रेंस स्वचालित डायलर, अनुस्मारक सूचनाएं और रिकॉर्ड मीटिंग जैसी कई उन्नत सुविधाएँ प्रदान करता है। सभी कॉल पासवर्ड से सुरक्षित हैं। एसएमएस और ईमेल एकीकरण कुशल अधिसूचना क्षमता प्रदान करता है। अधिक जानकारी के लिए यात्रा लिंक,<a href='https://www.c-zentrix.com/products/cz-conferencing-bridge'>यहाँ क्लिक करें</a>


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2227: UserWarning: `max_length` is ignored when `padding`=`True`.
  warnings.warn("`max_length` is ignored when `padding`=`True`.")
